# Datentransformation Credit Card

Ziel dieser Datenaufbereitung ist es alle Variablen so in einen Datensatz zu überführen.

In [1]:
from pathlib import Path
from scipy import stats

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

np.set_printoptions(suppress=True)

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = None

from IPython.display import display, Markdown

In [2]:
path1 = Path(r"A:\Workspace\Python\Masterarbeit\Kaggle Home Credit Datensatz")
path2 = Path(r"C:\Users\rober\Documents\Workspace\Python\Masterarbeit\Kaggle Home Credit Datensatz")

if path1.is_dir():
    DATASET_DIR = path1
else:
    DATASET_DIR = path2

In [3]:
app_train = pd.read_csv(DATASET_DIR / "application_train.csv")
credit_card_balance = pd.read_csv(DATASET_DIR / "credit_card_balance.csv")
des = pd.read_csv(DATASET_DIR / "HomeCredit_columns_description.csv", encoding="latin", index_col=0)

In [4]:
todo = list(credit_card_balance.columns)

In [5]:
credit_card_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,1800.0,1800.0,0.000,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,2250.0,2250.0,60175.080,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,2250.0,2250.0,26926.425,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,11925.0,11925.0,224949.285,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,27000.0,27000.0,443044.395,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


#### Keine Datenaufbereitung für Primär- und Fremdschlüssel

In [6]:
keys = ["SK_ID_CURR", "SK_ID_PREV"]

In [7]:
todo = [element for element in todo if element not in keys]

#### Datenaufbereitung metrische Daten

In [8]:
m_heads = [element for element in todo if credit_card_balance[element].dtype.name != "object"]

##### Gruppierung

In [9]:
quants = [0, 0.25, 0.5, 0.75, 1]

In [10]:
for index, head in enumerate(m_heads):
    
    df = credit_card_balance[["SK_ID_CURR", head]]
    df = df.groupby(by = ["SK_ID_CURR"]).quantile(quants)
    df = df.unstack(level=-1)
    
    try:
        mets = mets.join(df , on = "SK_ID_CURR")
    except (ValueError, NameError):
        mets = df

In [11]:
# Löschen der metrischen Daten aus der TODO Liste
todo = [element for element in todo if element not in m_heads]

In [12]:
mets.head()

MONTHS_BALANCE                          AMT_BALANCE            \
                     0.00   0.25  0.50   0.75 1.00        0.00 0.25 0.50   
SK_ID_CURR                                                                 
100006               -6.0  -4.75  -3.5  -2.25 -1.0         0.0  0.0  0.0   
100011              -75.0 -56.75 -38.5 -20.25 -2.0         0.0  0.0  0.0   
100013              -96.0 -72.25 -48.5 -24.75 -1.0         0.0  0.0  0.0   
100021              -18.0 -14.00 -10.0  -6.00 -2.0         0.0  0.0  0.0   
100023              -11.0  -9.25  -7.5  -5.75 -4.0         0.0  0.0  0.0   

                                   AMT_CREDIT_LIMIT_ACTUAL            \
                   0.75       1.00                    0.00      0.25   
SK_ID_CURR                                                             
100006           0.0000       0.00                270000.0  270000.0   
100011      116686.2825  189000.00                 90000.0  180000.0   
100013           0.0000  161420.22                 45000.0  157500.0   
100021           0.0000       0.00                675000.0  675000.0   
100023           0.0000       0.00                 45000.0   45000.0   

                                         AMT_DRAWINGS_ATM_CURRENT            \
                0.50      0.75      1.00                     0.00 0.25 0.50   
SK_ID_CURR                                                                    
100006      270000.0  270000.0  270000.0                      NaN  NaN  NaN   
100011      180000.0  180000.0  180000.0                      0.0  0.0  0.0   
100013      157500.0  157500.0  157500.0                      0.0  0.0  0.0   
100021      675000.0  675000.0  675000.0                      NaN  NaN  NaN   
100023      135000.0  225000.0  225000.0                      NaN  NaN  NaN   

                          AMT_DRAWINGS_CURRENT                           \
           0.75      1.00                 0.00 0.25 0.50 0.75      1.00   
SK_ID_CURR                                                                
100006      NaN       NaN                  0.0  0.0  0.0  0.0       0.0   
100011      0.0  180000.0                  0.0  0.0  0.0  0.0  180000.0   
100013      0.0  157500.0                  0.0  0.0  0.0  0.0  157500.0   
100021      NaN       NaN                  0.0  0.0  0.0  0.0       0.0   
100023      NaN       NaN                  0.0  0.0  0.0  0.0       0.0   

           AMT_DRAWINGS_OTHER_CURRENT                      \
                                 0.00 0.25 0.50 0.75 1.00   
SK_ID_CURR                                                  
100006                            NaN  NaN  NaN  NaN  NaN   
100011                            0.0  0.0  0.0  0.0  0.0   
100013                            0.0  0.0  0.0  0.0  0.0   
100021                            NaN  NaN  NaN  NaN  NaN   
100023                            NaN  NaN  NaN  NaN  NaN   

           AMT_DRAWINGS_POS_CURRENT                      \
                               0.00 0.25 0.50 0.75 1.00   
SK_ID_CURR                                                
100006                          NaN  NaN  NaN  NaN  NaN   
100011                          0.0  0.0  0.0  0.0  0.0   
100013                          0.0  0.0  0.0  0.0  0.0   
100021                          NaN  NaN  NaN  NaN  NaN   
100023                          NaN  NaN  NaN  NaN  NaN   

           AMT_INST_MIN_REGULARITY                            \
                              0.00 0.25 0.50    0.75    1.00   
SK_ID_CURR                                                     
100006                         0.0  0.0  0.0     0.0     0.0   
100011                         0.0  0.0  0.0  9000.0  9000.0   
100013                         0.0  0.0  0.0     0.0  7875.0   
100021                         0.0  0.0  0.0     0.0     0.0   
100023                         0.0  0.0  0.0     0.0     0.0   

           AMT_PAYMENT_CURRENT                                          \
                          0.00     0.25   

#### Datenaufbereitung kategorische Daten

In [13]:
cat_heads = [element for element in todo if credit_card_balance[element].dtype.name == "object"]

In [14]:
for index, head in enumerate(cat_heads):
    
    df = credit_card_balance[["SK_ID_CURR", head]]
    df = df.groupby(["SK_ID_CURR", head]).size()
    df = df.unstack()
    
    try:
        cats = cats.join(a , on="SK_ID_CURR")
    except (ValueError, NameError):
        cats = df

In [15]:
# Löschen der metrischen Daten aus der TODO Liste
todo = [element for element in todo if element not in cat_heads]

In [16]:
cats.head()

NAME_CONTRACT_STATUS,Active,Approved,Completed,Demand,Refused,Sent proposal,Signed
SK_ID_CURR,,,,,,,
100006,6.0,NaN,NaN,NaN,NaN,NaN,NaN
100011,74.0,NaN,NaN,NaN,NaN,NaN,NaN
100013,96.0,NaN,NaN,NaN,NaN,NaN,NaN
100021,7.0,NaN,10.0,NaN,NaN,NaN,NaN
100023,8.0,NaN,NaN,NaN,NaN,NaN,NaN


#### Zusammenführung metrischer und kategorischer Daten

In [17]:
credit_card_balance = mets.join(cats, on = "SK_ID_CURR")

c:\users\rober\documents\workspace\python\masterarbeit\venv\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


In [18]:
credit_card_balance.head()

,"(MONTHS_BALANCE, 0.0)","(MONTHS_BALANCE, 0.25)","(MONTHS_BALANCE, 0.5)","(MONTHS_BALANCE, 0.75)","(MONTHS_BALANCE, 1.0)","(AMT_BALANCE, 0.0)","(AMT_BALANCE, 0.25)","(AMT_BALANCE, 0.5)","(AMT_BALANCE, 0.75)","(AMT_BALANCE, 1.0)","(AMT_CREDIT_LIMIT_ACTUAL, 0.0)","(AMT_CREDIT_LIMIT_ACTUAL, 0.25)","(AMT_CREDIT_LIMIT_ACTUAL, 0.5)","(AMT_CREDIT_LIMIT_ACTUAL, 0.75)","(AMT_CREDIT_LIMIT_ACTUAL, 1.0)","(AMT_DRAWINGS_ATM_CURRENT, 0.0)","(AMT_DRAWINGS_ATM_CURRENT, 0.25)","(AMT_DRAWINGS_ATM_CURRENT, 0.5)","(AMT_DRAWINGS_ATM_CURRENT, 0.75)","(AMT_DRAWINGS_ATM_CURRENT, 1.0)","(AMT_DRAWINGS_CURRENT, 0.0)","(AMT_DRAWINGS_CURRENT, 0.25)","(AMT_DRAWINGS_CURRENT, 0.5)","(AMT_DRAWINGS_CURRENT, 0.75)","(AMT_DRAWINGS_CURRENT, 1.0)","(AMT_DRAWINGS_OTHER_CURRENT, 0.0)","(AMT_DRAWINGS_OTHER_CURRENT, 0.25)","(AMT_DRAWINGS_OTHER_CURRENT, 0.5)","(AMT_DRAWINGS_OTHER_CURRENT, 0.75)","(AMT_DRAWINGS_OTHER_CURRENT, 1.0)","(AMT_DRAWINGS_POS_CURRENT, 0.0)","(AMT_DRAWINGS_POS_CURRENT, 0.25)","(AMT_DRAWINGS_POS_CURRENT, 0.5)","(AMT_DRAWINGS_POS_CURRENT, 0.75)","(AMT_DRAWINGS_POS_CURRENT, 1.0)","(AMT_INST_MIN_REGULARITY, 0.0)","(AMT_INST_MIN_REGULARITY, 0.25)","(AMT_INST_MIN_REGULARITY, 0.5)","(AMT_INST_MIN_REGULARITY, 0.75)","(AMT_INST_MIN_REGULARITY, 1.0)","(AMT_PAYMENT_CURRENT, 0.0)","(AMT_PAYMENT_CURRENT, 0.25)","(AMT_PAYMENT_CURRENT, 0.5)","(AMT_PAYMENT_CURRENT, 0.75)","(AMT_PAYMENT_CURRENT, 1.0)","(AMT_PAYMENT_TOTAL_CURRENT, 0.0)","(AMT_PAYMENT_TOTAL_CURRENT, 0.25)","(AMT_PAYMENT_TOTAL_CURRENT, 0.5)","(AMT_PAYMENT_TOTAL_CURRENT, 0.75)","(AMT_PAYMENT_TOTAL_CURRENT, 1.0)","(AMT_RECEIVABLE_PRINCIPAL, 0.0)","(AMT_RECEIVABLE_PRINCIPAL, 0.25)","(AMT_RECEIVABLE_PRINCIPAL, 0.5)","(AMT_RECEIVABLE_PRINCIPAL, 0.75)","(AMT_RECEIVABLE_PRINCIPAL, 1.0)","(AMT_RECIVABLE, 0.0)","(AMT_RECIVABLE, 0.25)","(AMT_RECIVABLE, 0.5)","(AMT_RECIVABLE, 0.75)","(AMT_RECIVABLE, 1.0)","(AMT_TOTAL_RECEIVABLE, 0.0)","(AMT_TOTAL_RECEIVABLE, 0.25)","(AMT_TOTAL_RECEIVABLE, 0.5)","(AMT_TOTAL_RECEIVABLE, 0.75)","(AMT_TOTAL_RECEIVABLE, 1.0)","(CNT_DRAWINGS_ATM_CURRENT, 0.0)","(CNT_DRAWINGS_ATM_CURRENT, 0.25)","(CNT_DRAWINGS_ATM_CURRENT, 0.5)","(CNT_DRAWINGS_ATM_CURRENT, 0.75)","(CNT_DRAWINGS_ATM_CURRENT, 1.0)","(CNT_DRAWINGS_CURRENT, 0.0)","(CNT_DRAWINGS_CURRENT, 0.25)","(CNT_DRAWINGS_CURRENT, 0.5)","(CNT_DRAWINGS_CURRENT, 0.75)","(CNT_DRAWINGS_CURRENT, 1.0)","(CNT_DRAWINGS_OTHER_CURRENT, 0.0)","(CNT_DRAWINGS_OTHER_CURRENT, 0.25)","(CNT_DRAWINGS_OTHER_CURRENT, 0.5)","(CNT_DRAWINGS_OTHER_CURRENT, 0.75)","(CNT_DRAWINGS_OTHER_CURRENT, 1.0)","(CNT_DRAWINGS_POS_CURRENT, 0.0)","(CNT_DRAWINGS_POS_CURRENT, 0.25)","(CNT_DRAWINGS_POS_CURRENT, 0.5)","(CNT_DRAWINGS_POS_CURRENT, 0.75)","(CNT_DRAWINGS_POS_CURRENT, 1.0)","(CNT_INSTALMENT_MATURE_CUM, 0.0)","(CNT_INSTALMENT_MATURE_CUM, 0.25)","(CNT_INSTALMENT_MATURE_CUM, 0.5)","(CNT_INSTALMENT_MATURE_CUM, 0.75)","(CNT_INSTALMENT_MATURE_CUM, 1.0)","(SK_DPD, 0.0)","(SK_DPD, 0.25)","(SK_DPD, 0.5)","(SK_DPD, 0.75)","(SK_DPD, 1.0)","(SK_DPD_DEF, 0.0)","(SK_DPD_DEF, 0.25)","(SK_DPD_DEF, 0.5)","(SK_DPD_DEF, 0.75)","(SK_DPD_DEF, 1.0)",Active,Approved,Completed,Demand,Refused,Sent proposal,Signed
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100006,-6.0,-4.75,-3.5,-2.25,-1.0,0.0,0.0,0.0,0.0000,0.00,270000.0,270000.0,270000.0,270000.0,270000.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.000,0.0,0.0,0.0000,0.00,0.000,0.0,0.0,0.0000,0.00,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN
100011,-75.0,-56.75,-38.5,-20.25,-2.0,0.0,0.0,0.0,116686.2825,189000.00,90000.0,180000.0,180000.0,180000.0,180000.0,0.0,0.0,0.0,0.0,180000.0,0.0,0.0,0.0,0.0,180000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9000.0,9000.0,0.0,563.355,563.355,9000.00000,55485.0,0.0,0.0,0.0,9000.

### Abspeichern der Daten

In [19]:
credit_card_balance.to_csv(DATASET_DIR / "Datengruppierung" / "credit_card_balance.csv")

In [20]:
todo

[]